In [1]:
using ForwardDiff

In [2]:
function F(xu::Vector, h = 0.01, d = 4) 
    x, y, theta, v, w, a = xu
#     w, a = u
    
    f = h*v
    b = f*cos(w) + d - sqrt(d^2 - f^2 * sin(w)^2)

    x = x + b*cos(theta)
    y = y + b*sin(theta)
    theta = theta + asin(sin(w) * f/d)
    v = v + h*a
    
    return [x, y, theta, v]
    
end

F (generic function with 3 methods)

In [3]:
z(x, p) = sqrt(x^2 + p^2) - p

z (generic function with 1 method)

In [4]:
function L(x_u, p = [0.1, 0.1, 0.01, 1] , c_w = 0.01, c_a = 0.0001)
    x, y, theta, v, w, a = x_u
    p_x, p_y, p_theta, p_v = p
    0.01(z(x, p_x) + z(y, p_y)) + c_w * w^2 + c_a * a^2
end

L (generic function with 4 methods)

In [37]:
Lf(x, p = [0.1, 0.1, 0.01, 1], weight = 100000) =
    weight * (z(x[1], p[1]) + z(x[2], p[2]) + z(x[3], p[3]) + z(x[4], p[4]))

Lf (generic function with 3 methods)

In [38]:
ForwardDiff.gradient(Lf, [1, 2, 3, 4])

4-element Array{Float64,1}:
 99503.71902099892
 99875.23388778446
 99999.44444907403
 97014.25001453319

In [21]:
function j(U, x0 = [1, 1, 3/2 * pi, 0], N = 5)#total loss
    x = x0
    for i in 1:N
        x = F(vcat(x0, U[2i-1:2i]))
    end 
    L(vcat(x, U[2N:2N+1]))
#         L(vcat(F(vcat(x0, U)), U))
    
end

j (generic function with 3 methods)

In [31]:
function j(U, x0 = [1, 1, 3/2 * pi, 0]; N = 0)# loss of (x_N, u_N)
    x = x0
    for i in 1:N
        x = F(vcat(x0, U[2i-1:2i]))
    end 
    L(vcat(x, U[2N+1:2N+2]))
#         L(vcat(F(vcat(x0, U)), U))
    
end

j (generic function with 3 methods)

In [39]:
function jf(U, x0 = [1, 1, 3/2 * pi, 0]; N = 100)
    x = x0
    for i in 1:N
        x = F(vcat(x0, U[2i-1:2i]))
    end
    Lf(x)
end

jf (generic function with 2 methods)

In [40]:
jf(1:200)

774844.2692426252

In [34]:
for i in 0:1
    println(i)
end

0
1


In [41]:
J(U; N = 100) = sum(j(U, N = i) for i in 0:N-1) + jf(U; N = N)

J (generic function with 1 method)

In [42]:
ForwardDiff.gradient(J, 1:200)

200-element Array{Float64,1}:
   0.02                 
   0.0004               
   0.06                 
   0.0008               
   0.1                  
   0.0012000000000000001
   0.14                 
   0.0016               
   0.18                 
   0.002                
   0.22                 
   0.0024000000000000002
   0.26                 
   ⋮                    
   3.7800000000000002   
   0.038                
   3.8200000000000003   
   0.038400000000000004 
   3.86                 
   0.0388               
   3.9                  
   0.0392               
   3.94                 
   0.0396               
   3.98                 
 894.4671909999159      

In [36]:
ForwardDiff.gradient(J, 1:12)

12-element Array{Float64,1}:
 0.02                 
 0.0004               
 0.06                 
 0.0008               
 0.1                  
 0.0012000000000000001
 0.14                 
 0.0016               
 0.18                 
 0.002                
 0.22                 
 0.0024000000000000002